# NLP basic challenges

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

## Word Frequency Counter 📊 Challenge:
Write a Python function that takes a text document as input and returns a dictionary containing the frequency of each word in the document.

### 1-basic approach

In [2]:
import re

def count_doc(doc_path):
    vocab = {}
    with open(doc_path, 'r', encoding='utf-8') as doc:
        for line in doc:
            # Remove punctuation and convert to lowercase
            words = re.findall(r'\b\w+\b', line.lower())
            for word in words:
                if word not in vocab:
                    vocab[word] = 1
                else:
                    vocab[word] += 1
    return vocab


### 2- map-reduce approach

### **Concept de MapReduce :**
MapReduce est un paradigme de programmation utilisé pour traiter de grandes quantités de données de manière distribuée. Il se divise en deux étapes principales :

1. **Map** : 
   - Divise le travail en tâches indépendantes.
   - Transforme les données d'entrée en paires clé-valeur `(clé, valeur)`.
   - Exemples : Compter les mots, générer `(mot, 1)` pour chaque mot trouvé.

2. **Reduce** :
   - Regroupe les paires ayant la même clé.
   - Applique une opération (comme une somme, une moyenne, etc.) pour réduire plusieurs valeurs associées à une clé en une seule.
   - Exemples : Additionner les valeurs pour obtenir la fréquence totale de chaque mot.

---

### **Étapes de MapReduce :**
1. **Split** :
   - Diviser les données d'entrée en morceaux (partitions).
   - Exemple : Un fichier texte de 1 To divisé en blocs de 64 Mo.

2. **Map** :
   - Traiter chaque morceau de manière indépendante pour générer des paires clé-valeur.
   - Exemple : Depuis un texte, produire `(mot, 1)` pour chaque mot.

3. **Shuffle and Sort** :
   - Regrouper et trier les données par clé (toutes les occurrences de la même clé ensemble).
   - Exemple : Regrouper tous les `(mot, 1)` par mot.

4. **Reduce** :
   - Effectuer une opération sur les groupes de clés triés.
   - Exemple : Calculer la somme des valeurs pour chaque clé `(mot, total)`.

5. **Output** :
   - Produire les résultats finaux.
   - Exemple : Une liste contenant chaque mot et sa fréquence.

---

### **Illustration avec un exemple (comptage de mots) :**
#### Entrée :
```
Texte : "Bonjour monde, bonjour Python"
```

#### Étapes :
1. **Map** :
   ```
   Bonjour -> (bonjour, 1)
   monde -> (monde, 1)
   bonjour -> (bonjour, 1)
   Python -> (python, 1)
   ```

2. **Shuffle and Sort** :
   ```
   (bonjour, [1, 1]), (monde, [1]), (python, [1])
   ```

3. **Reduce** :
   ```
   (bonjour, 2), (monde, 1), (python, 1)
   ```

#### Résultat final :
```
{'bonjour': 2, 'monde': 1, 'python': 1}
```

---

### **Avantages :**
- **Scalabilité** : Idéal pour traiter des données massives sur plusieurs machines.
- **Parallélisme** : Les étapes Map et Reduce peuvent être parallélisées.

### **Limitation** :
- Pas adapté à tous les problèmes (par exemple, ceux nécessitant de fortes dépendances entre données).

In [3]:
import re

def map_words(doc_path):
    with open(doc_path, 'r', encoding='utf-8') as doc:
        for line in doc:
            words = re.findall(r'\b\w+\b', line.lower())  # Clean and tokenize
            for word in words:
                yield (word, 1)  # Yield each word with a count of 1

# Reduce function: aggregates counts for each word
def reduce_word_counts(mapped_words):
    word_counts = {}
    for word, count in mapped_words:
        if word not in word_counts:
            word_counts[word] += 1  # Sum up counts for each word
        else:
            word_counts[word] += count
    return word_counts


# Combine Map and Reduce
def map_reduce_word_count(doc_path):
    mapped_words = map_words(doc_path)  # Step 1: Map
    reduced_counts = reduce_word_counts(mapped_words)  # Step 2: Reduce
    return reduced_counts

## Text Cleaning and Tokenization 🧹 Challenge:

Create a function to clean and tokenize a given text, removing punctuation and converting words to lowercase

In [3]:
test = "hello, world"
test.split()

['hello,', 'world']

In [4]:
def clean_text(text):
    # lower the text
    text = text.lower()
    # remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # tokenize
    return text.split()

> re.sub(r'[^\w\s]', '', text) :

Supprime tous les caractères qui ne sont ni alphanumériques (\w) ni des espaces (\s).

> re.sub(r'\s+', ' ', text).strip() :

Remplace plusieurs espaces par un seul, puis enlève les espaces superflus au début ou à la fin.

## Stopword Removal 🚫 Challenge

Develop a function that removes stopwords (commonly used words) from a given sentence or text.

In [5]:
import nltk
from nltk.corpus import stopwords

# Téléchargement des stop words (à faire une seule fois)
nltk.download('stopwords')

def remove_stop_words(text):
    # Définir la liste des stop words en anglais
    stop_words = set(stopwords.words('english'))
    # Supprimer les stop words du texte
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text


ModuleNotFoundError: No module named 'nltk'

## Sentiment Analysis 🌟 Challenge:

In [ ]:
import nltk
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# Download required NLTK datasets
nltk.download('movie_reviews')
nltk.download('stopwords')

# Load the dataset
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

# Shuffle the dataset
import random
random.shuffle(documents)

# Preprocess text (remove stop words and convert to lowercase)
stop_words = set(stopwords.words('english'))

def preprocess(words):
    return ' '.join([word.lower() for word in words if word.lower() not in stop_words])

# Prepare the data
texts = [preprocess(words) for words, label in documents]
labels = [label for _, label in documents]

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=42)

# Convert text data into numerical features using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to top 5000 features
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a classifier (e.g., Naive Bayes)
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred))


Voici les étapes principales du processus **TF-IDF (Term Frequency-Inverse Document Frequency)** :

1. **Calcul du Term Frequency (TF)** :
   - Le **TF** mesure la fréquence d'apparition d'un mot dans un document par rapport à l'ensemble du document. 
   - Formellement : 
     \[
     TF(t, d) = \frac{\text{Nombre d'occurrences du terme } t \text{ dans le document } d}{\text{Nombre total de termes dans le document } d}
     \]

2. **Calcul de l'Inverse Document Frequency (IDF)** :
   - L'**IDF** mesure l'importance d'un terme dans l'ensemble des documents. Plus un mot apparaît dans peu de documents, plus il est important.
   - Formellement :
     \[
     IDF(t) = \log\left(\frac{\text{Nombre total de documents}}{\text{Nombre de documents contenant le terme } t}\right)
     \]

3. **Calcul du TF-IDF** :
   - Le **TF-IDF** est le produit du **TF** et du **IDF**, ce qui permet de mesurer l'importance d'un terme dans un document par rapport à l'ensemble du corpus.
   - Formellement :
     \[
     TFIDF(t, d) = TF(t, d) \times IDF(t)
     \]

---

### **Résumé du processus** :
1. Calculez le **TF** pour chaque mot dans chaque document.
2. Calculez le **IDF** pour chaque mot dans l'ensemble des documents.
3. Multipliez les valeurs **TF** et **IDF** pour chaque mot dans chaque document pour obtenir le **TF-IDF**.

Le **TF-IDF** donne plus de poids aux mots fréquents dans un document, mais rares dans l'ensemble du corpus, ce qui permet de capturer les termes significatifs.

## Named Entity Recognition (NER) 🧐 Challenge:
Implement a Named Entity Recognition algorithm that identifies and classifies named entities like names, locations, and organizations in a text.

## Text Similarity 📜 Challenge: 
Create a function that measures the similarity between two text documents using techniques like TF-IDF or cosine similarity.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Function to clean the text (optional step to remove special characters)
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Function to compute similarity between two documents
def compute_similarity(doc1, doc2):
    # Clean the documents (optional)
    doc1 = clean_text(doc1)
    doc2 = clean_text(doc2)
    
    # Initialize TF-IDF Vectorizer
    tfidf_vectorizer = TfidfVectorizer()
    
    # Fit and transform the documents
    tfidf_matrix = tfidf_vectorizer.fit_transform([doc1, doc2])
    
    # Compute cosine similarity between the two document vectors
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    
    return cosine_sim[0][0]  # Cosine similarity score between 0 and 1

# Example usage
doc1 = "I love programming in Python. It's a great language for data science."
doc2 = "Python is awesome for data science and machine learning tasks."

similarity = compute_similarity(doc1, doc2)
print(f"Cosine Similarity: {similarity}")


ModuleNotFoundError: No module named 'sklearn'

## Topic Modeling 📌 Challenge: 
Use techniques like Latent Dirichlet Allocation (LDA) to perform topic modeling on a collection of documents.

Le **Latent Dirichlet Allocation (LDA)** est un modèle statistique utilisé pour la **modélisation de sujets** dans des corpus de textes. Il permet de découvrir des structures latentes (sujets) dans un ensemble de documents sans supervision, en considérant que chaque document est une combinaison de plusieurs sujets. Voici un résumé des étapes et du concept d'**LDA** :

---

### **Concept de LDA**
LDA est un modèle de **génération de texte** qui suppose que chaque document est un mélange de plusieurs sujets, et que chaque mot dans un document est associé à un sujet spécifique. L'objectif de LDA est de **découvrir** ces sujets latents en analysant la distribution des mots dans les documents.

#### **Hypothèses principales** :
1. Chaque document est constitué d’un **mélange** de sujets (thèmes).
2. Chaque sujet est une distribution sur les mots dans le vocabulaire.
3. LDA cherche à découvrir la répartition de ces sujets et mots à partir des données observées.

---

### **Étapes du processus LDA**

1. **Initialisation** :
   - Chaque mot d'un document est attribué aléatoirement à un sujet.
   - On initialise les distributions de sujets pour chaque document et les distributions de mots pour chaque sujet.

2. **Estimation des paramètres** :
   - LDA utilise une **approche bayésienne** pour estimer les paramètres du modèle. L'idée est d'itérer sur le processus de **réaffectation** des mots à un sujet et de mettre à jour les distributions de manière à maximiser la vraisemblance des données.
   
   Les deux principales distributions sont :
   - \( \theta_d \) : La distribution des sujets pour le document \( d \).
   - \( \phi_k \) : La distribution des mots pour le sujet \( k \).

3. **Mise à jour des affectations** :
   - À chaque itération, chaque mot est réaffecté à un sujet en fonction de deux critères :
     - La probabilité que ce mot vienne du sujet, basée sur la distribution de mots de chaque sujet.
     - La probabilité que ce document soit associé à ce sujet, basée sur la distribution de sujets de ce document.

4. **Répétition jusqu'à convergence** :
   - Ce processus est répété plusieurs fois, ajustant continuellement les affectations de mots et les distributions des sujets jusqu'à ce que les distributions se stabilisent.

5. **Finalisation du modèle** :
   - Une fois l'algorithme convergé, chaque document peut être caractérisé par une distribution de sujets, et chaque sujet peut être décrit par une distribution de mots.

---

### **Résumé du processus LDA** :
1. **Suppositions** : Chaque document est un mélange de sujets et chaque sujet est une distribution sur les mots.
2. **Initialisation** : Affectation aléatoire des mots aux sujets.
3. **Itération** :
   - Mise à jour des affectations des mots aux sujets.
   - Mise à jour des distributions des sujets et des mots.
4. **Convergence** : L'algorithme converge vers un modèle stable.

---

### **Application de LDA** :
- **Découverte de sujets** dans un corpus de texte.
- **Réduction de dimension** pour des tâches comme la classification ou la recommandation.
- **Compréhension du contenu** en extrayant des thèmes ou sujets sous-jacents.

### **Exemple d'Application en Python (avec Gensim)** :
```python
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

nltk.download('punkt')
nltk.download('stopwords')

# Exemple de documents
documents = ["Machine learning is fascinating.", 
             "Natural language processing is a subfield of AI.",
             "Deep learning is a subset of machine learning.",
             "I love studying algorithms in computer science."]

# Prétraitement des documents
stop_words = set(stopwords.words('english'))
processed_docs = [[word for word in word_tokenize(doc.lower()) if word.isalpha() and word not in stop_words] 
                  for doc in documents]

# Créer un dictionnaire
dictionary = corpora.Dictionary(processed_docs)

# Créer le corpus
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# Appliquer LDA
lda_model = gensim.models.LdaMulticore(corpus, num_topics=2, id2word=dictionary, passes=10)

# Afficher les topics
topics = lda_model.print_topics(num_words=3)
for topic in topics:
    print(topic)
```

---

### **Avantages et Limites de LDA** :
- **Avantages** :
  - Identifie des sujets latents dans un corpus de textes sans supervision.
  - Utile pour l'analyse de contenu à grande échelle (extraction de thèmes).
  
- **Limites** :
  - Nécessite de spécifier le nombre de sujets à l'avance.
  - La qualité des sujets dépend fortement de la prétraitement des données et de la sélection des hyperparamètres.

LDA est un outil puissant pour explorer des corpus textuels et extraire des informations significatives sur les thèmes sous-jacents, mais il faut parfois expérimenter avec les hyperparamètres pour obtenir des résultats optimaux.